In [1]:
import pandas as pd
import numpy as np
import re
import librosa
from tqdm import tqdm
import os

In [2]:
canciones = pd.read_csv('/content/drive/MyDrive/Bootcamp_DS/ML/IA/train.csv')

In [3]:
canciones

,song_id,filename,filepath,genre_id,genre
0,10150,010150.ogg,train/010150.ogg,7,Instrumental
1,7358,007358.ogg,train/007358.ogg,2,Punk
2,20573,020573.ogg,train/020573.ogg,5,Folk
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic
4,16662,016662.ogg,train/016662.ogg,1,Rock
...,...,...,...,...,...
19917,9337,009337.ogg,train/009337.ogg,0,Electronic
19918,8340,008340.ogg,train/008340.ogg,4,Hip-Hop
19919,16248,016248.ogg,train/016248.ogg,4,Hip-Hop
19920,11875,011875.ogg,train/011875.ogg,2,Punk


In [4]:
# Se sustituye el .ogg por .wav

canciones["filepath"] = canciones["filepath"].str.replace(".ogg", ".wav")

In [5]:
# Se cambia el file path para que coincida con la estructura de carpetas

canciones["filepath"] = canciones.apply(lambda row: row["filepath"].replace("train", str(row["genre_id"])), axis=1)

In [6]:
# Se eliminan los 0 entre / y el primer numero para que coincida exactamente con la estructura de carpetas

canciones["filepath"] = canciones["filepath"].apply(lambda x: re.sub(r"/0+", "/", x))

In [15]:
canciones[canciones["filepath"] == "0/.wav"]

,song_id,filename,filepath,genre_id,genre
16360,0,000000.ogg,0/.wav,0,Electronic


In [24]:
canciones.iloc[16360].values

array([0, '000000.ogg', '0/.wav', 0, 'Electronic'], dtype=object)

In [27]:
canciones.loc[16360] = [0, '000000.ogg', '0/0.wav', 0, 'Electronic']

In [28]:
canciones.loc[16360][2]

<ipython-input-28-d08fa6457e6e>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  canciones.loc[16360][2]


'0/0.wav'

In [15]:
def extract_mfcc(file_path, n_mfcc=20):
    try:
        # Cargar el archivo de audio
        y, sr = librosa.load(file_path, sr=None)

        # Extraer MFCC
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)

        # Calcular la media de cada coeficiente MFCC
        mfcc_means = np.mean(mfccs, axis=1)

        return mfcc_means
    except Exception as e:
        print(f"Error procesando {file_path}: {e}")
        return np.full(n_mfcc, np.nan)



In [ ]:
def extract_audio_features(file_path, n_mfcc=20):
    try:
        y, sr = librosa.load(file_path, sr=None)

        # Chroma
        chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)

        # Spectral Contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr), axis=1)

        # Zero Crossing Rate
        zero_crossing = np.mean(librosa.feature.zero_crossing_rate(y), axis=1)

        # Spectral Rolloff
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr), axis=1)

        # Tempo (BPM)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

        # Agregar el tempo a las características
        return np.hstack([chroma, spectral_contrast, zero_crossing, rolloff, tempo])

    except Exception as e:
        print(f"Error procesando {file_path}: {e}")
        return np.full(12 + 7 + 1 + 1 + 1, np.nan)  # Rellenar con NaN en caso de error


In [31]:
AUDIO_BASE_PATH = "/content/drive/MyDrive/Bootcamp_DS/ML/TRAIN_V2/data_out_2/"

In [ ]:
mfcc_features = []
for _, row in tqdm(canciones.iterrows(), total=len(canciones)):
    file_path = os.path.join(AUDIO_BASE_PATH, row["filepath"])
    mfcc = extract_mfcc(file_path)
    mfcc_features.append(mfcc)

  7%|▋         | 1359/19922 [08:38<1:55:25,  2.68it/s]<ipython-input-7-3e42b1a4357d>:4: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error procesando /content/drive/MyDrive/Bootcamp_DS/ML/TRAIN_V2/data_out_2/1/3137.wav: [Errno 2] No such file or directory: '/content/drive/MyDrive/Bootcamp_DS/ML/TRAIN_V2/data_out_2/1/3137.wav'


 23%|██▎       | 4668/19922 [28:20<1:25:26,  2.98it/s]

Error procesando /content/drive/MyDrive/Bootcamp_DS/ML/TRAIN_V2/data_out_2/2/11088.wav: [Errno 2] No such file or directory: '/content/drive/MyDrive/Bootcamp_DS/ML/TRAIN_V2/data_out_2/2/11088.wav'


 26%|██▌       | 5150/19922 [31:15<1:32:18,  2.67it/s]

...

 93%|█████████▎| 18537/19922 [1:51:23<07:20,  3.14it/s]

Error procesando /content/drive/MyDrive/Bootcamp_DS/ML/TRAIN_V2/data_out_2/2/3071.wav: [Errno 2] No such file or directory: '/content/drive/MyDrive/Bootcamp_DS/ML/TRAIN_V2/data_out_2/2/3071.wav'


 94%|█████████▍| 18796/19922 [1:52:53<06:09,  3.05it/s]

Error procesando /content/drive/MyDrive/Bootcamp_DS/ML/TRAIN_V2/data_out_2/4/13954.wav: [Errno 2] No such file or directory: '/content/drive/MyDrive/Bootcamp_DS/ML/TRAIN_V2/data_out_2/4/13954.wav'


100%|██████████| 19922/19922 [2:00:47<00:00,  2.75it/s]


In [ ]:
mfcc_df = pd.DataFrame(mfcc_features, columns=[f"MFCC_{i+1}" for i in range(20)])

# Concatenar con el dataset original
canciones = pd.concat([canciones, mfcc_df], axis=1)

# Guardar el nuevo dataset con MFCCs en CSV
canciones.to_csv("/content/drive/MyDrive/Bootcamp_DS/ML/IA/train_with_mfcc.csv", index=False)

# También guardarlo como un archivo .npy más eficiente
np.save("/content/drive/MyDrive/Bootcamp_DS/ML/IAtrain_mfcc.npy", mfcc_df.to_numpy())

print("¡Procesamiento de MFCC completado y guardado!")

¡Procesamiento de MFCC completado y guardado!


In [32]:
features = []
for _, row in tqdm(canciones.iterrows(), total=len(canciones)):
    file_path = os.path.join(AUDIO_BASE_PATH, row["filepath"])
    audio_features = extract_audio_features(file_path)
    features.append(audio_features)

  3%|▎         | 607/19922 [06:07<3:15:08,  1.65it/s]


KeyboardInterrupt: 

In [ ]:
df_canciones = pd.read_csv('/content/drive/MyDrive/Bootcamp_DS/ML/IA/train_with_mfcc.csv')

In [ ]:
features_df = pd.DataFrame(features, columns=[f"Chroma_{i+1}" for i in range(12)] + [f"Spectral_contrast_{i+1}" for i in range(7)] + ["Zero_crossing_rate", "Spectral_Rolloff", "Tempo"])

# Concatenar con el dataset original
canciones = pd.concat([df_canciones, features_df], axis=1)

# Guardar el nuevo dataset con MFCCs en CSV
canciones.to_csv("/content/drive/MyDrive/Bootcamp_DS/ML/IA/train_with_features.csv", index=False)

# También guardarlo como un archivo .npy más eficiente
np.save("/content/drive/MyDrive/Bootcamp_DS/ML/IA/train_features.npy", features_df.to_numpy())

print("¡Procesamiento de Features completado y guardado!")

In [ ]:
canciones.head()

,song_id,filename,filepath,genre_id,genre,MFCC_1,MFCC_2,MFCC_3,MFCC_4,MFCC_5,...,MFCC_11,MFCC_12,MFCC_13,MFCC_14,MFCC_15,MFCC_16,MFCC_17,MFCC_18,MFCC_19,MFCC_20
0,10150,010150.ogg,7/10150.wav,7,Instrumental,-211.066559,100.593956,42.598457,23.165283,2.860841,...,1.383807,-0.954104,-0.424609,2.930789,1.142149,-5.260527,3.911721,-3.118353,3.531898,4.903169
1,7358,007358.ogg,2/7358.wav,2,Punk,-14.467098,99.263069,-2.664984,26.246733,2.537334,...,-3.182864,2.670346,-4.157828,-1.427059,-5.992962,-2.554197,-8.692637,-0.044264,-7.512719,-0.651457
2,20573,020573.ogg,5/20573.wav,5,Folk,-317.806274,90.202530,-36.330082,18.486454,-25.418125,...,-18.698023,-9.699656,-11.490185,-17.160065,-15.421208,-11.540291,6.645436,8.064914,3.898278,3.883015
3,11170,011170.ogg,12/11170.wav,12,Old-Time / Historic,-264.162598,143.274414,-109.257156,-26.363281,-6.620979,...,-14.553981,-5.271422,-13.570509,-0.129010,8.321633,-9.815311,15.549283,17.530434,-11.841340,6.789664
4,16662,016662.ogg,1/16662.wav,1,Rock,2.097910,85.692604,-1.235375,24.334158,1.336215,...,-6.963677,4.131877,-3.015029,-1.620542,-4.801994,-0.453704,-6.365971,1.639646,-2.654665,2.383693


In [2]:
df_canciones = pd.read_csv('/content/drive/MyDrive/Bootcamp_DS/ML/IA/train_with_mfcc.csv')

In [12]:
df_canciones.loc[16360, "filepath"] = "0/0.wav"

In [19]:
df_canciones.loc[16360].values

array([0, '000000.ogg', '0/0.wav', 0, 'Electronic', nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan], dtype=object)

In [18]:
mfcc_0 = extract_mfcc("/content/drive/MyDrive/Bootcamp_DS/ML/TRAIN_V2/data_out_2/0/0.wav")

In [21]:
mfcc_0

array([-181.71925  ,  133.34796  ,   14.8828945,   26.577473 ,
          8.056044 ,   15.998042 ,   -4.473025 ,    4.1229315,
          0.8465411,    3.4274905,   -7.135882 ,   -1.218502 ,
         -2.9580157,    1.9757385,  -11.066191 ,    1.1861942,
         -7.8885903,    2.1857052,   -5.7356305,   -5.1370807],
      dtype=float32)

In [22]:
df_canciones.loc[16360] = [0, '000000.ogg', '0/0.wav', 0, 'Electronic', -181.71925  ,  133.34796  ,   14.8828945,   26.577473 ,
          8.056044 ,   15.998042 ,   -4.473025 ,    4.1229315,
          0.8465411,    3.4274905,   -7.135882 ,   -1.218502 ,
         -2.9580157,    1.9757385,  -11.066191 ,    1.1861942,
         -7.8885903,    2.1857052,   -5.7356305,   -5.1370807]

In [23]:
df_canciones.loc[16360]

,16360
song_id,0
filename,000000.ogg
filepath,0/0.wav
genre_id,0
genre,Electronic
MFCC_1,-181.71925
MFCC_2,133.34796
MFCC_3,14.882895
MFCC_4,26.577473
MFCC_5,8.056044


In [30]:
df_canciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19922 entries, 0 to 19921
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   song_id   19922 non-null  int64  
 1   filename  19922 non-null  object 
 2   filepath  19922 non-null  object 
 3   genre_id  19922 non-null  int64  
 4   genre     19922 non-null  object 
 5   MFCC_1    19909 non-null  float64
 6   MFCC_2    19909 non-null  float64
 7   MFCC_3    19909 non-null  float64
 8   MFCC_4    19909 non-null  float64
 9   MFCC_5    19909 non-null  float64
 10  MFCC_6    19909 non-null  float64
 11  MFCC_7    19909 non-null  float64
 12  MFCC_8    19909 non-null  float64
 13  MFCC_9    19909 non-null  float64
 14  MFCC_10   19909 non-null  float64
 15  MFCC_11   19909 non-null  float64
 16  MFCC_12   19909 non-null  float64
 17  MFCC_13   19909 non-null  float64
 18  MFCC_14   19909 non-null  float64
 19  MFCC_15   19909 non-null  float64
 20  MFCC_16   19909 non-null  fl

In [31]:
df_canciones[df_canciones["MFCC_1"].isnull()]["genre_id"].value_counts().sort_index()

,count
genre_id,
0,2
1,2
2,2
3,1
4,4
5,1
7,1


In [32]:
df_canciones["genre_id"].value_counts().sort_index()

,count
genre_id,
0,3073
1,3097
2,2584
3,1801
4,1761
5,1215
6,1181
7,1045
8,945


In [34]:
df_canciones.dropna(inplace=True)

In [35]:
df_canciones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19909 entries, 0 to 19921
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   song_id   19909 non-null  int64  
 1   filename  19909 non-null  object 
 2   filepath  19909 non-null  object 
 3   genre_id  19909 non-null  int64  
 4   genre     19909 non-null  object 
 5   MFCC_1    19909 non-null  float64
 6   MFCC_2    19909 non-null  float64
 7   MFCC_3    19909 non-null  float64
 8   MFCC_4    19909 non-null  float64
 9   MFCC_5    19909 non-null  float64
 10  MFCC_6    19909 non-null  float64
 11  MFCC_7    19909 non-null  float64
 12  MFCC_8    19909 non-null  float64
 13  MFCC_9    19909 non-null  float64
 14  MFCC_10   19909 non-null  float64
 15  MFCC_11   19909 non-null  float64
 16  MFCC_12   19909 non-null  float64
 17  MFCC_13   19909 non-null  float64
 18  MFCC_14   19909 non-null  float64
 19  MFCC_15   19909 non-null  float64
 20  MFCC_16   19909 non-null  float64

In [36]:
df_canciones.to_csv("/content/drive/MyDrive/Bootcamp_DS/ML/IA/train_with_mfcc.csv", index=False)
